In [1]:
# Import required libraries
import random  # For random sampling of data
import ast  # For safely evaluating strings containing Python literals
import pandas as pd  # For data manipulation and analysis with DataFrames
from datasets import load_dataset  # For loading datasets from Hugging Face
from transformers import pipeline  # For using pre-trained transformer models
from senticnet.senticnet import SenticNet  # For accessing SenticNet knowledge graph
from sklearn.metrics import classification_report, confusion_matrix  # For model evaluation metrics

# ---------------------------
# 1. Load dataset
# ---------------------------
dataset = load_dataset("NEUDM/semeval-2014", "laptops")  # Load SemEval 2014 laptops dataset
train_data = dataset["train"]  # Extract the training split from the dataset

# ---------------------------
# 2. Initialize SenticNet
# ---------------------------
sn = SenticNet()  # Create SenticNet object to access concept polarity values

# ---------------------------
# 3. Load BERT sentiment pipeline
# ---------------------------
bert_pipeline = pipeline(
    "sentiment-analysis",  # Specify task as sentiment analysis
    model="nlptown/bert-base-multilingual-uncased-sentiment",  # Use pre-trained BERT model
    device=-1  # Use CPU for inference (GPU not available)
)

# ---------------------------
# 4. Helper functions
# ---------------------------
def stars_to_label(star_label):
    # Convert star ratings to sentiment labels
    if "1 star" in star_label or "2 star" in star_label:  # 1-2 stars = negative
        return "negative"
    elif "3 star" in star_label:  # 3 stars = neutral
        return "neutral"
    elif "4 star" in star_label or "5 star" in star_label:  # 4-5 stars = positive
        return "positive"
    return "neutral"  # Default fallback to neutral

def bert_predict(sentence):
    # Get BERT prediction for a sentence and convert to sentiment label
    return stars_to_label(bert_pipeline(sentence)[0]["label"])  # Call pipeline and extract label

def bert_kg_predict(sentence, aspects):
    """Predict using BERT and adjust with SenticNet for each aspect."""
    pred = stars_to_label(bert_pipeline(sentence)[0]["label"])  # Get initial BERT prediction
    
    for aspect in aspects:  # Iterate through each aspect in the sentence
        for word in aspect.lower().split():  # Split aspect into individual words
            try:
                polarity = float(sn.polarity_value(word))  # Get SenticNet polarity for word
                if polarity > 0:  # Positive polarity overrides prediction
                    pred = "positive"
                elif polarity < 0:  # Negative polarity overrides prediction
                    pred = "negative"
                else:  # Neutral polarity overrides prediction
                    pred = "neutral"
            except KeyError:  # Handle words not found in SenticNet
                continue  # Skip to next word if current word not found
    return pred  # Return final adjusted prediction

# ---------------------------
# 5. Aspect categorization
# ---------------------------
def categorize_aspect(aspect):
    aspect = aspect.lower()  # Convert aspect to lowercase for case-insensitive matching
    if any(x in aspect for x in ["battery", "keyboard", "screen", "display", "memory", "cpu"]):
        return "Hardware"  # Categorize as Hardware if contains hardware-related terms
    elif any(x in aspect for x in ["speed", "performance", "responsiveness", "reliability"]):
        return "Performance"  # Categorize as Performance if contains performance terms
    elif any(x in aspect for x in ["price", "cost", "value"]):
        return "Price"  # Categorize as Price if contains price-related terms
    elif any(x in aspect for x in ["warranty", "support", "service", "delivery"]):
        return "Support"  # Categorize as Support if contains support terms
    else:
        return "Other"  # Default category for unmatched aspects

# ---------------------------
# 6. Select random 1000 sentences
# ---------------------------
random.seed(42)  # Set random seed for reproducible sampling
selected_indices = random.sample(range(len(train_data)), 1000)  # Randomly select 1000 indices
selected_rows = [train_data[i] for i in selected_indices]  # Get actual data for selected indices

# ---------------------------
# 7. Run experiment
# ---------------------------
results = []  # Initialize empty list to store experiment results

for row in selected_rows:  # Iterate through each selected data row
    sentence = row["input"]  # Extract the review sentence/text
    
    # Parse aspects from the output field
    aspects_raw = ast.literal_eval(row["output"]) if isinstance(row["output"], str) else row["output"]
    aspects_list = []  # Initialize list to store (aspect, sentiment) pairs

    # Handle different output formats (dict or list)
    if isinstance(aspects_raw, dict):  # If output is dictionary format
        for key in aspects_raw:  # Iterate through dictionary keys
            for item in aspects_raw[key]:  # Iterate through items in each key
                if item[0]:  # Check if aspect exists (not empty)
                    aspects_list.append((item[0], item[1]))  # Add (aspect, sentiment) tuple
    elif isinstance(aspects_raw, list):  # If output is list format
        for item in aspects_raw:  # Iterate through list items
            if item[0]:  # Check if aspect exists (not empty)
                aspects_list.append((item[0], item[1]))  # Add (aspect, sentiment) tuple

    if not aspects_list:  # Skip if no valid aspects found
        continue
    
    # Get predictions from both models for the sentence
    bert_pred_single = bert_predict(sentence)  # BERT-only prediction
    bertkg_pred_single = bert_kg_predict(sentence, [a for a,_ in aspects_list])  # BERT+KG prediction
    
    for asp, gt in aspects_list:  # Iterate through each aspect and its ground truth
        # Determine which model(s) predicted correctly
        if bert_pred_single == gt and bertkg_pred_single == gt:
            case = "Both Correct"  # Both models correct
        elif bert_pred_single == gt and bertkg_pred_single != gt:
            case = "BERT Correct Only"  # Only BERT correct
        elif bert_pred_single != gt and bertkg_pred_single == gt:
            case = "BERT+KG Correct Only"  # Only BERT+KG correct
        else:
            case = "Both Wrong"  # Both models incorrect
        
        # Store results for this aspect
        results.append({
            "Sentence": sentence,  # Original sentence
            "Aspect": asp,  # Aspect term
            "Category": categorize_aspect(asp),  # Categorized aspect
            "Ground Truth": gt,  # Actual sentiment label
            "BERT": bert_pred_single,  # BERT prediction
            "BERT+KG": bertkg_pred_single,  # BERT+KG prediction
            "Case": case  # Which model(s) were correct
        })

# ---------------------------
# 8. Create dataframe
# ---------------------------
df = pd.DataFrame(results)  # Convert results list to pandas DataFrame
print(f"Total aspects analyzed: {len(df)}\n")  # Print total number of aspects processed
print(df.head(15))  # Display first 15 rows of the results

# ---------------------------
# 9. Numeric evaluation per category
# ---------------------------
categories = df["Category"].unique()  # Get unique aspect categories
labels = ["negative","neutral","positive"]  # Define sentiment labels for evaluation

for cat in categories:  # Iterate through each category
    df_cat = df[df["Category"] == cat]  # Filter DataFrame for current category
    if df_cat.empty:  # Skip if no data for this category
        continue
    
    print(f"\n===== Evaluation for {cat} =====")  # Print category header
    
    y_true = df_cat["Ground Truth"]  # Extract ground truth labels
    y_pred_bert = df_cat["BERT"]  # Extract BERT predictions
    y_pred_bertkg = df_cat["BERT+KG"]  # Extract BERT+KG predictions
    
    # BERT evaluation
    print("\n--- BERT ---")  # Print BERT section header
    print(confusion_matrix(y_true, y_pred_bert, labels=labels))  # Print confusion matrix
    print(classification_report(y_true, y_pred_bert, labels=labels, zero_division=0))  # Print classification report
    
    # BERT+KG evaluation
    print("\n--- BERT+KG ---")  # Print BERT+KG section header
    print(confusion_matrix(y_true, y_pred_bertkg, labels=labels))  # Print confusion matrix
    print(classification_report(y_true, y_pred_bertkg, labels=labels, zero_division=0))  # Print classification report

# ---------------------------
# 10. Case-based analysis summary
# ---------------------------
print("\n===== Case-based Comparison =====")  # Print case analysis header
case_summary = df.groupby(["Category", "Case"]).size().unstack(fill_value=0)  # Group by category and case, count occurrences
print(case_summary)  # Print the case summary table

Device set to use cpu


Total aspects analyzed: 1713

                                             Sentence  \
0   [So having the AC plug go out on me and get lo...   
1   [, Applications respond immediately (not like ...   
2   [, Applications respond immediately (not like ...   
3   [Price no more than a Jersey deli but way bett...   
4   [The biggest problem is that the box had no in...   
5   [Only thing I would want to add to this is an ...   
6              [The battery gets so HOT it is scary.]   
7   [However, the multi-touch gestures and large t...   
8   [However, the multi-touch gestures and large t...   
9   [However, the multi-touch gestures and large t...   
10  [However, the multi-touch gestures and large t...   
11  [I took my girlfriend there for her birthday l...   
12  [The AMD Turin processor seems to always perfo...   
13  [The AMD Turin processor seems to always perfo...   
14  [There is a downside if you're ordering in -- ...   

                     Aspect  Category Ground Truth      B

In [ ]:
# Import required libraries for evaluation and visualization
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report  # For model evaluation metrics and visualization
import matplotlib.pyplot as plt  # For creating plots and charts

# Define the sentiment classes used in the analysis
labels = ["negative", "neutral", "positive"]  # define the 3 sentiment classes

# Extract the true labels and predictions from the DataFrame
y_true = df["Ground Truth"]  # Get the actual sentiment labels (ground truth)
y_pred_bert = df["BERT"]  # Get the predictions from BERT model
y_pred_bertkg = df["BERT+KG"]  # Get the predictions from BERT+Knowledge Graph model

# ---------------------------
# Confusion matrix for BERT
# ---------------------------
# Create confusion matrix comparing true labels vs BERT predictions
cm_bert = confusion_matrix(y_true, y_pred_bert, labels=labels)
# Create a display object for the confusion matrix with proper labels
disp_bert = ConfusionMatrixDisplay(confusion_matrix=cm_bert, display_labels=labels)
# Plot the confusion matrix with blue color scheme
disp_bert.plot(cmap="Blues")
# Add title to the plot
plt.title("Confusion Matrix - BERT")
# Display the plot
plt.show()

# Print detailed classification report for BERT model
print("BERT Classification Report")
print(classification_report(
    y_true,  # True labels
    y_pred_bert,  # Predicted labels from BERT
    labels=labels,  # Class labels to include
    target_names=labels,  # Names for the classes in reports
    zero_division=0  # Set value when division by zero occurs
))

# ---------------------------
# Confusion matrix for BERT+KG
# ---------------------------
# Create confusion matrix comparing true labels vs BERT+KG predictions
cm_bertkg = confusion_matrix(y_true, y_pred_bertkg, labels=labels)
# Create a display object for the confusion matrix with proper labels
disp_bertkg = ConfusionMatrixDisplay(confusion_matrix=cm_bertkg, display_labels=labels)
# Plot the confusion matrix with green color scheme
disp_bertkg.plot(cmap="Greens")
# Add title to the plot
plt.title("Confusion Matrix - BERT+KG")
# Display the plot
plt.show()

# Print detailed classification report for BERT+KG model
print("BERT+KG Classification Report")
print(classification_report(
    y_true,  # True labels
    y_pred_bertkg,  # Predicted labels from BERT+KG
    labels=labels,  # Class labels to include
    target_names=labels,  # Names for the classes in reports
    zero_division=0  # Set value when division by zero occurs
))

# ---------------------------
# Case-based analysis (who wins where)
# ---------------------------
# Initialize empty list to store case analysis results
cases = []
# Iterate through each true label and corresponding predictions from both models
for gt, b, kg in zip(y_true, y_pred_bert, y_pred_bertkg):
    # Check if both models predicted correctly
    if b == gt and kg == gt:
        cases.append("Both Correct")  # Both models are correct
    # Check if only BERT predicted correctly
    elif b == gt and kg != gt:
        cases.append("BERT Correct Only")  # Only BERT is correct
    # Check if only BERT+KG predicted correctly
    elif b != gt and kg == gt:
        cases.append("BERT+KG Correct Only")  # Only BERT+KG is correct
    # Both models predicted incorrectly
    else:
        cases.append("Both Wrong")  # Both models are wrong

# Add the case analysis results as a new column to the DataFrame
df["Case"] = cases

# Print summary of case-based comparison
print("\n===== Case-based Comparison =====")
# Count occurrences of each case type and display
print(df["Case"].value_counts())

# Optional: visualize case distribution
# Create a bar chart showing the distribution of different cases
df["Case"].value_counts().plot(kind="bar", color=["skyblue","orange","green","red"])
# Add title to the bar chart
plt.title("Case-based Comparison of BERT vs BERT+KG")
# Add label to y-axis
plt.ylabel("Number of Aspects")
# Display the bar chart
plt.show()

In [ ]:
# ---------------------------
# Inspect cases where BERT is correct but BERT+KG fails
# ---------------------------
# Filter the DataFrame to get only rows where BERT was correct but BERT+KG was wrong
bert_only = df[df["Case"] == "BERT Correct Only"]
# Print header for this section
print("=== BERT Correct Only ===")
# Display the first 20 rows with specific columns to analyze these cases
print(bert_only[["Sentence", "Aspect", "Ground Truth", "BERT", "BERT+KG"]].head(20))
# Shows examples where BERT's prediction was right but adding knowledge graph made it wrong

# ---------------------------
# Inspect cases where BERT+KG is correct but BERT fails
# ---------------------------
# Filter the DataFrame to get only rows where BERT+KG was correct but BERT was wrong
bertkg_only = df[df["Case"] == "BERT+KG Correct Only"]
# Print header for this section
print("=== BERT+KG Correct Only ===")
# Display the first 20 rows with specific columns to analyze these cases
print(bertkg_only[["Sentence", "Aspect", "Ground Truth", "BERT", "BERT+KG"]].head(20))
# Shows examples where the knowledge graph helped correct BERT's mistakes

# ---------------------------
# Inspect cases where both fail
# ---------------------------
# Filter the DataFrame to get only rows where both models predicted incorrectly
both_wrong = df[df["Case"] == "Both Wrong"]
# Print header for this section
print("=== Both Wrong ===")
# Display the first 20 rows with specific columns to analyze these cases
print(both_wrong[["Sentence", "Aspect", "Ground Truth", "BERT", "BERT+KG"]].head(20))
# Shows challenging cases that neither approach could handle correctly

In [ ]:
# Import required libraries for plotting and evaluation metrics
import matplotlib.pyplot as plt  # For creating visualizations and plots
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # For model evaluation and confusion matrix visualization

# Define the sentiment class labels used in the analysis
labels = ["negative", "neutral", "positive"]  # The three sentiment categories for classification

# ---------------------------
# Confusion matrix for BERT
# ---------------------------
# Create confusion matrix comparing true labels vs BERT predictions
cm_bert = confusion_matrix(y_true, y_pred_bert, labels=labels)
# Create a display object for visualizing the confusion matrix
disp_bert = ConfusionMatrixDisplay(confusion_matrix=cm_bert, display_labels=labels)
# Plot the confusion matrix with blue color scheme and integer formatting
disp_bert.plot(cmap="Blues", values_format="d")
# Add title to the confusion matrix plot
plt.title("Confusion Matrix - BERT")
# Adjust layout to prevent label cutoff
plt.tight_layout()
# Save the plot as high-quality PDF (good for LaTeX documents)
plt.savefig("cm_bert.pdf")
# Also save as high-resolution PNG (300 DPI) for other uses
plt.savefig("cm_bert.png", dpi=300)
# Close the current figure to free memory and prepare for next plot
plt.close()

# ---------------------------
# Confusion matrix for BERT+KG
# ---------------------------
# Create confusion matrix comparing true labels vs BERT+KG predictions
cm_bertkg = confusion_matrix(y_true, y_pred_bertkg, labels=labels)
# Create a display object for visualizing the confusion matrix
disp_bertkg = ConfusionMatrixDisplay(confusion_matrix=cm_bertkg, display_labels=labels)
# Plot the confusion matrix with green color scheme and integer formatting
disp_bertkg.plot(cmap="Greens", values_format="d")
# Add title to the confusion matrix plot
plt.title("Confusion Matrix - BERT+KG")
# Adjust layout to prevent label cutoff
plt.tight_layout()
# Save the plot as high-quality PDF (good for LaTeX documents)
plt.savefig("cm_bertkg.pdf")
# Also save as high-resolution PNG (300 DPI) for other uses
plt.savefig("cm_bertkg.png", dpi=300)
# Close the current figure to free memory and prepare for next plot
plt.close()

# ---------------------------
# Case-based comparison bar chart
# ---------------------------
# Count the number of occurrences for each case type
case_counts = df["Case"].value_counts()
# Create a bar chart showing the distribution of different cases with colored bars
ax = case_counts.plot(kind="bar", color=["skyblue","orange","green","red"])
# Add title to the bar chart
plt.title("Case-based Comparison of BERT vs BERT+KG")
# Add label to the y-axis indicating what the numbers represent
plt.ylabel("Number of Aspects")
# Rotate x-axis labels 30 degrees and align them to the right for better readability
plt.xticks(rotation=30, ha="right")
# Adjust layout to prevent label cutoff
plt.tight_layout()
# Save the bar chart as high-quality PDF (good for LaTeX documents)
plt.savefig("case_distribution.pdf")
# Also save as high-resolution PNG (300 DPI) for other uses
plt.savefig("case_distribution.png", dpi=300)
# Close the current figure to free memory
plt.close()

In [ ]:
# Import required libraries for evaluation metrics, visualization, and data handling
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report  # For model evaluation and metrics
import matplotlib.pyplot as plt  # For creating plots and visualizations
import pandas as pd  # For data manipulation and CSV export

# Define the sentiment class labels used in the classification task
labels = ["negative", "neutral", "positive"]  # define the 3 sentiment classes

# Extract the true labels and model predictions from the DataFrame
y_true = df["Ground Truth"]  # Get the actual sentiment labels (ground truth values)
y_pred_bert = df["BERT"]  # Get the predictions from the BERT model
y_pred_bertkg = df["BERT+KG"]  # Get the predictions from the BERT+Knowledge Graph model

# ---------------------------
# Confusion matrix for BERT
# ---------------------------
# Create confusion matrix comparing true labels vs BERT predictions
cm_bert = confusion_matrix(y_true, y_pred_bert, labels=labels)
# Create visualization object for the confusion matrix
disp_bert = ConfusionMatrixDisplay(confusion_matrix=cm_bert, display_labels=labels)
# Plot the confusion matrix with blue color scheme
disp_bert.plot(cmap="Blues")
# Add title to the plot
plt.title("Confusion Matrix - BERT")
# Save the confusion matrix plot as PNG image file
plt.savefig("confusion_matrix_bert.png")
# Close the current figure to free memory
plt.close()

# Generate detailed classification report for BERT model and return as dictionary
report_bert = classification_report(
    y_true,  # True labels
    y_pred_bert,  # BERT predictions
    labels=labels,  # Class labels to include
    target_names=labels,  # Names for display in report
    zero_division=0,  # Value to use when there's division by zero
    output_dict=True  # Return results as dictionary instead of string
)

# Save classification report to CSV file for further analysis
# Convert dictionary to DataFrame, transpose it, and save as CSV
pd.DataFrame(report_bert).transpose().to_csv("classification_report_bert.csv")

# ---------------------------
# Confusion matrix for BERT+KG
# ---------------------------
# Create confusion matrix comparing true labels vs BERT+KG predictions
cm_bertkg = confusion_matrix(y_true, y_pred_bertkg, labels=labels)
# Create visualization object for the confusion matrix
disp_bertkg = ConfusionMatrixDisplay(confusion_matrix=cm_bertkg, display_labels=labels)
# Plot the confusion matrix with green color scheme
disp_bertkg.plot(cmap="Greens")
# Add title to the plot
plt.title("Confusion Matrix - BERT+KG")
# Save the confusion matrix plot as PNG image file
plt.savefig("confusion_matrix_bertkg.png")
# Close the current figure to free memory
plt.close()

# Generate detailed classification report for BERT+KG model and return as dictionary
report_bertkg = classification_report(
    y_true,  # True labels
    y_pred_bertkg,  # BERT+KG predictions
    labels=labels,  # Class labels to include
    target_names=labels,  # Names for display in report
    zero_division=0,  # Value to use when there's division by zero
    output_dict=True  # Return results as dictionary instead of string
)

# Save classification report to CSV file for further analysis
# Convert dictionary to DataFrame, transpose it, and save as CSV
pd.DataFrame(report_bertkg).transpose().to_csv("classification_report_bertkg.csv")

# ---------------------------
# Case-based analysis
# ---------------------------
# Initialize empty list to store case analysis results
cases = []
# Iterate through each true label and corresponding predictions from both models
for gt, b, kg in zip(y_true, y_pred_bert, y_pred_bertkg):
    # Check if both models predicted correctly
    if b == gt and kg == gt:
        cases.append("Both Correct")  # Both models are correct
    # Check if only BERT predicted correctly
    elif b == gt and kg != gt:
        cases.append("BERT Correct Only")  # Only BERT is correct
    # Check if only BERT+KG predicted correctly
    elif b != gt and kg == gt:
        cases.append("BERT+KG Correct Only")  # Only BERT+KG is correct
    # Both models predicted incorrectly
    else:
        cases.append("Both Wrong")  # Both models are wrong

# Add the case analysis results as a new column to the DataFrame
df["Case"] = cases

# Save the complete DataFrame with case analysis to CSV file
# index=False prevents saving the row index as a separate column
df.to_csv("df_with_cases.csv", index=False)

# Create and save case distribution visualization
# Count the number of occurrences for each case type
case_counts = df["Case"].value_counts()
# Create a bar chart showing the distribution of different cases
case_counts.plot(kind="bar", color=["skyblue","orange","green","red"])
# Add title to the bar chart
plt.title("Case-based Comparison of BERT vs BERT+KG")
# Add label to the y-axis
plt.ylabel("Number of Aspects")
# Save the case comparison plot as PNG image file
plt.savefig("case_comparison.png")
# Close the current figure to free memory
plt.close()

In [ ]:
# Import accuracy and F1 score metrics from scikit-learn
from sklearn.metrics import accuracy_score, f1_score

# Extract the true labels and model predictions from the DataFrame
y_true = df["Ground Truth"]  # Get the actual sentiment labels (ground truth values)
y_pred_bert = df["BERT"]  # Get the predictions from the BERT model
y_pred_bertkg = df["BERT+KG"]  # Get the predictions from the BERT+Knowledge Graph model

# Calculate accuracy scores for both models
# Accuracy = (Number of correct predictions) / (Total number of predictions)
acc_bert = accuracy_score(y_true, y_pred_bert)  # Calculate accuracy for BERT model
acc_bertkg = accuracy_score(y_true, y_pred_bertkg)  # Calculate accuracy for BERT+KG model

# Print the accuracy results for both models
print("BERT Accuracy:", acc_bert)  # Display BERT accuracy score
print("BERT+KG Accuracy:", acc_bertkg)  # Display BERT+KG accuracy score

# Calculate Macro F1 scores for both models
# Macro F1 = average of F1 scores for each class (equal weight for all classes)
macro_f1_bert = f1_score(y_true, y_pred_bert, average='macro')  # Calculate macro F1 for BERT
macro_f1_bertkg = f1_score(y_true, y_pred_bertkg, average='macro')  # Calculate macro F1 for BERT+KG

# Print the Macro F1 results for both models
print("BERT Macro-F1:", macro_f1_bert)  # Display BERT macro F1 score
print("BERT+KG Macro-F1:", macro_f1_bertkg)  # Display BERT+KG macro F1 score

# Calculate Micro F1 scores for both models
# Micro F1 = global F1 score considering total true positives, false negatives, false positives
micro_f1_bert = f1_score(y_true, y_pred_bert, average='micro')  # Calculate micro F1 for BERT
micro_f1_bertkg = f1_score(y_true, y_pred_bertkg, average='micro')  # Calculate micro F1 for BERT+KG

# Print the Micro F1 results for both models
print("BERT Micro-F1:", micro_f1_bert)  # Display BERT micro F1 score
print("BERT+KG Micro-F1:", micro_f1_bertkg)  # Display BERT+KG micro F1 score